In [7]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import label_binarize
from utility import Dataset, oh_encoder, t_encoder, l_encoder, scaler, get_best_features, cfs, rfe, sfs, pca, eval_metric
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, chi2, f_classif, mutual_info_classif

from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from catboost import CatBoostClassifier, Pool, metrics, cv
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix

from sklearn.metrics import roc_curve, auc

columns = ['duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes', 'land', 'wrong_fragment',
           'urgent', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell', 'su_attempted',
           'num_root', 'num_file_creations', 'num_shells', 'num_access_files', 'num_outbound_cmds', 'is_host_login',
           'is_guest_login', 'count', 'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate',
           'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count',
           'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate',
           'dst_host_srv_diff_host_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate', 'dst_host_rerror_rate',
           'dst_host_srv_rerror_rate', 'label', 'score']

nominal_features = ['protocol_type', 'service', 'flag']
binary_features = ['land', 'logged_in', 'root_shell', 'su_attempted', 'is_host_login', 'is_guest_login']
numeric_features = [feature for feature in columns if feature not in nominal_features + binary_features + ['label', 'score', 'num_outbound_cmds']]

starting_train_df = pd.read_csv(f'dataset/nsl-kdd/KDDTrain+.txt', header=None)
starting_test_df = pd.read_csv(f'dataset/nsl-kdd/KDDTest+.txt', header=None)

def pipeline(train_df, test_df):
    models = {
        'Logistic Regression': LogisticRegression(),
        'Random Forest': RandomForestClassifier(),
        'Decision Tree': DecisionTreeClassifier(),
        'LinearSVC': LinearSVC(),
        'GaussianNB': GaussianNB()
    }

    y_train = train_df['label']
    y_test = test_df['label']
    X_train = train_df.drop(columns=['label'])
    X_test = test_df.drop(columns=['label'])

    n_features = X_train.shape[1]
    for k in range(1, n_features + 1):
        for method in [chi2, f_classif, mutual_info_classif]:
            print(f"Selecting {k} best features using {method.__name__}...")
            X_train, X_test = get_best_features(train_df, test_df, method, k)
            y_train = train_df['label']
            y_test = test_df['label']

            for name, model in models.items():
                print(f"Training {name}...")
                model.fit(X_train, y_train)
                y_pred = model.predict(X_test)
                print(classification_report(y_test, y_pred, target_names=model.classes_))


        X_train, X_test = rfe(train_df, test_df, k)
        print(f"Selecting {k} best features using RFE...")
        y_train = train_df['label']
        y_test = test_df['label']

        for name, model in models.items():
            print(f"Training {name}...")
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            print(classification_report(y_test, y_pred, target_names=model.classes_))


        X_train, X_test = pca(train_df, test_df, k)
        print(f"Selecting {k} best features using PCA...")
        y_train = train_df['label']
        y_test = test_df['label']

        for name, model in models.items():
            print(f"Training {name}...")
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            print(classification_report(y_test, y_pred, target_names=model.classes_))

    X_train, X_test = cfs(train_df, test_df)
    print(f"Selecting best features using CFS...")
    y_train = train_df['label']
    y_test = test_df['label']

    for name, model in models.items():
        print(f"Training {name}...")
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        print(classification_report(y_test, y_pred, target_names=model.classes_))

1. Con 2 label

In [8]:
train_df = Dataset(starting_train_df, columns).get_label2()
test_df = Dataset(starting_test_df, columns).get_label2()

1.1 Scalato

In [9]:
scaled_train_df, scaled_test_df = scaler(train_df, test_df, numeric_features, MinMaxScaler())

1.1.1 Solo one hot encoder

In [10]:
train_df, test_df = oh_encoder(scaled_train_df, scaled_test_df, ['protocol_type', 'flag', 'service'])

1.1.1.1 f_classif \
1.1.1.2 chi2 \
1.1.1.3 mutual_info_classif \
1.1.1.4 cfs \
1.1.1.5 rfe \
1.1.1.6 pca 

In [11]:
pipeline(train_df, test_df)

Selecting 1 best features using chi2...
Index(['dst_host_srv_serror_rate'], dtype='object')
Training Logistic Regression...
              precision    recall  f1-score   support

      attack       0.99      0.21      0.34     12833
      normal       0.49      1.00      0.65      9711

    accuracy                           0.55     22544
   macro avg       0.74      0.60      0.50     22544
weighted avg       0.77      0.55      0.48     22544

Training Random Forest...
              precision    recall  f1-score   support

      attack       0.99      0.15      0.26     12833
      normal       0.47      1.00      0.64      9711

    accuracy                           0.52     22544
   macro avg       0.73      0.57      0.45     22544
weighted avg       0.77      0.52      0.42     22544

Training Decision Tree...
              precision    recall  f1-score   support

      attack       0.99      0.16      0.27     12833
      normal       0.47      1.00      0.64      9711

    ac

c:\Users\crist\miniconda3\envs\envTesi\lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


              precision    recall  f1-score   support

      attack       0.99      0.19      0.33     12833
      normal       0.48      1.00      0.65      9711

    accuracy                           0.54     22544
   macro avg       0.74      0.60      0.49     22544
weighted avg       0.77      0.54      0.47     22544

Training GaussianNB...
              precision    recall  f1-score   support

      attack       0.99      0.21      0.34     12833
      normal       0.49      1.00      0.65      9711

    accuracy                           0.55     22544
   macro avg       0.74      0.60      0.50     22544
weighted avg       0.77      0.55      0.48     22544

Selecting 1 best features using f_classif...
Index(['flag_SF'], dtype='object')
Training Logistic Regression...
              precision    recall  f1-score   support

      attack       0.99      0.59      0.74     12833
      normal       0.65      0.99      0.78      9711

    accuracy                           0.76    

c:\Users\crist\miniconda3\envs\envTesi\lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


              precision    recall  f1-score   support

      attack       0.99      0.59      0.74     12833
      normal       0.65      0.99      0.78      9711

    accuracy                           0.76     22544
   macro avg       0.82      0.79      0.76     22544
weighted avg       0.84      0.76      0.76     22544

Training GaussianNB...
              precision    recall  f1-score   support

      attack       0.99      0.59      0.74     12833
      normal       0.65      0.99      0.78      9711

    accuracy                           0.76     22544
   macro avg       0.82      0.79      0.76     22544
weighted avg       0.84      0.76      0.76     22544

Selecting 1 best features using mutual_info_classif...
Index(['src_bytes'], dtype='object')
Training Logistic Regression...


...

c:\Users\crist\miniconda3\envs\envTesi\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


              precision    recall  f1-score   support

      attack       0.92      0.62      0.74     12833
      normal       0.65      0.93      0.76      9711

    accuracy                           0.75     22544
   macro avg       0.78      0.77      0.75     22544
weighted avg       0.80      0.75      0.75     22544

Training Random Forest...
              precision    recall  f1-score   support

      attack       0.97      0.64      0.77     12833
      normal       0.67      0.97      0.79      9711

    accuracy                           0.78     22544
   macro avg       0.82      0.81      0.78     22544
weighted avg       0.84      0.78      0.78     22544

Training Decision Tree...
              precision    recall  f1-score   support

      attack       0.92      0.67      0.77     12833
      normal       0.68      0.92      0.78      9711

    accuracy                           0.78     22544
   macro avg       0.80      0.79      0.78     22544
weighted avg       0.8

c:\Users\crist\miniconda3\envs\envTesi\lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


              precision    recall  f1-score   support

      attack       0.91      0.61      0.73     12833
      normal       0.64      0.93      0.76      9711

    accuracy                           0.74     22544
   macro avg       0.78      0.77      0.74     22544
weighted avg       0.80      0.74      0.74     22544

Training GaussianNB...
              precision    recall  f1-score   support

      attack       0.97      0.30      0.46     12833
      normal       0.52      0.99      0.68      9711

    accuracy                           0.60     22544
   macro avg       0.74      0.64      0.57     22544
weighted avg       0.78      0.60      0.55     22544



KeyboardInterrupt: 

1.1.2 Solo label encoder

In [ ]:
train_df, test_df = l_encoder(scaled_train_df, scaled_test_df, ['service', 'protocol_type', 'flag'])

1.1.2.1 f_classif \
1.1.2.2 chi2 \
1.1.2.3 mutual_info_classif \
1.1.2.4 cfs \
1.1.2.5 rfe \
1.1.2.6 pca 

In [ ]:
pipeline(train_df, test_df)

1.1.3 solo target encoder

In [ ]:
train_df, test_df = t_encoder(scaled_train_df, scaled_test_df, ['service', 'protocol_type', 'flag'])

1.1.3.1 f_classif \
1.1.3.2 chi2 \
1.1.3.3 mutual_info_classif \
1.1.3.4 cfs \
1.1.3.5 rfe \
1.1.3.6 pca 

In [ ]:
pipeline(train_df, test_df)

1.1.4 ohe + label

In [ ]:
train_df, test_df = oh_encoder(scaled_train_df, scaled_test_df, ['protocol_type', 'flag'])
train_df, test_df = l_encoder(train_df, test_df, ['service',])

1.1.4.1 f_classif \
1.1.4.2 chi2 \
1.1.4.3 mutual_info_classif \
1.1.4.4 cfs \
1.1.4.5 rfe \
1.1.4.6 pca 

In [ ]:
pipeline(train_df, test_df)

1.1.5 ohe + target

In [ ]:
train_df, test_df = t_encoder(scaled_train_df, scaled_test_df, ['service', ])
train_df, test_df = oh_encoder(train_df, test_df, ['protocol_type', 'flag'])

1.1.5.1 f_classif \
1.1.5.2 chi2 \
1.1.5.3 mutual_info_classif \
1.1.5.4 cfs \
1.1.5.5 rfe \
1.1.5.6 pca 

In [ ]:
pipeline(train_df, test_df)

1.2 Non scalato

In [ ]:
# DIOCANE
unscaled_train_df = Dataset(starting_train_df, columns).get_label2()
unscaled_test_df = Dataset(starting_test_df, columns).get_label2()

1.2.1 Solo one hot encoder

In [ ]:
train_df, test_df = oh_encoder(unscaled_train_df, unscaled_test_df, ['protocol_type', 'flag', 'service'])

1.2.1.1 f_classif \
1.2.1.2 chi2 \
1.2.1.3 mutual_info_classif \
1.2.1.4 cfs \
1.2.1.5 rfe \
1.2.1.6 pca 

In [ ]:
pipeline(train_df, test_df)

1.2.2 Solo label encoder

In [ ]:
train_df, test_df = l_encoder(unscaled_train_df, unscaled_test_df, ['service', 'protocol_type', 'flag'])

1.2.2.1 f_classif \
1.2.2.2 chi2 \
1.2.2.3 mutual_info_classif \
1.2.2.4 cfs \
1.2.2.5 rfe \
1.2.2.6 pca 

In [ ]:
pipeline(train_df, test_df)

1.2.3 solo target encoder

In [ ]:
train_df, test_df = t_encoder(unscaled_train_df, unscaled_test_df, ['service', 'protocol_type', 'flag'])

1.2.3.1 f_classif \
1.2.3.2 chi2 \
1.2.3.3 mutual_info_classif \
1.2.3.4 cfs \
1.2.3.5 rfe \
1.2.3.6 pca 

In [ ]:
pipeline(train_df, test_df)

1.2.4 ohe + label

In [ ]:
train_df, test_df = oh_encoder(unscaled_train_df, unscaled_test_df, ['protocol_type', 'flag'])
train_df, test_df = l_encoder(train_df, test_df, ['service', ])

1.2.4.1 f_classif \
1.2.4.2 chi2 \
1.2.4.3 mutual_info_classif \
1.2.4.4 cfs \
1.2.4.5 rfe \
1.2.4.6 pca 

In [ ]:
pipeline(train_df, test_df)

1.2.5 ohe + target

In [ ]:
train_df, test_df = t_encoder(unscaled_train_df, unscaled_test_df, ['service', ])
train_df, test_df = oh_encoder(train_df, test_df, ['protocol_type', 'flag'])

1.2.5.1 f_classif \
1.2.5.2 chi2 \
1.2.5.3 mutual_info_classif \
1.2.5.4 cfs \
1.2.5.5 rfe \
1.2.5.6 pca 

In [ ]:
pipeline(train_df, test_df)

1.3 Discretizzato

In [ ]:
train_df = Dataset(starting_train_df, columns).get_label2()
test_df = Dataset(starting_test_df, columns).get_label2()

discretizer = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='uniform')
X_train = train_df.drop(columns=['label'])
X_test = test_df.drop(columns=['label'])
X_train = discretizer.fit_transform(X_train)
X_test = discretizer.transform(X_test)

discretized_train_df = pd.concat([pd.DataFrame(X_train), train_df['label']], axis=1)
discretized_test_df = pd.concat([pd.DataFrame(X_test), test_df['label']], axis=1)

1.3.1 Solo one hot encoder

In [ ]:
train_df, test_df = oh_encoder(discretized_train_df, discretized_test_df, ['protocol_type', 'flag', 'service'])

1.1.3.1 f_classif \
1.1.3.2 chi2 \
1.1.3.3 mutual_info_classif \
1.1.3.4 cfs \
1.1.3.5 rfe \
1.1.3.6 pca 

In [ ]:
pipeline(train_df, test_df)

1.3.2 Solo label encoder

In [ ]:
train_df, test_df = l_encoder(discretized_train_df, discretized_test_df, ['service', 'protocol_type', 'flag'])

1.3.2.1 f_classif \
1.3.2.2 chi2 \
1.3.2.3 mutual_info_classif \
1.3.2.4 cfs \
1.3.2.5 rfe \
1.3.2.6 pca 

In [ ]:
pipeline(train_df, test_df)

1.3.3 solo target encoder

In [ ]:
train_df, test_df = t_encoder(discretized_train_df, discretized_test_df, ['service', 'protocol_type', 'flag'])

1.3.3.1 f_classif \
1.3.3.2 chi2 \
1.3.3.3 mutual_info_classif \
1.3.3.4 cfs \
1.3.3.5 rfe \
1.3.3.6 pca 

In [ ]:
pipeline(train_df, test_df)

1.3.4 ohe + label

In [ ]:
train_df, test_df = oh_encoder(discretized_train_df, discretized_test_df, ['protocol_type', 'flag'])
train_df, test_df = l_encoder(train_df, test_df, ['service', ])

1.3.4.1 f_classif \
1.3.4.2 chi2 \
1.3.4.3 mutual_info_classif \
1.3.4.4 cfs \
1.3.4.5 rfe \
1.3.4.6 pca 

In [ ]:
pipeline(train_df, test_df)

1.3.5 ohe + target

In [ ]:
train_df, test_df = t_encoder(discretized_train_df, discretized_test_df, ['service', ])
train_df, test_df = oh_encoder(train_df, test_df, ['protocol_type', 'flag'])

1.3.5.1 f_classif \
1.3.5.2 chi2 \
1.3.5.3 mutual_info_classif \
1.3.5.4 cfs \
1.3.5.5 rfe \
1.3.5.6 pca 

In [ ]:
pipeline(train_df, test_df)

2. Con 5 label

In [ ]:
train_df = Dataset(starting_train_df, columns).get_label5()
test_df = Dataset(starting_test_df, columns).get_label5()

2.1 Scalato

In [ ]:
scaled_train_df, scaled_test_df = scaler(train_df, test_df, numeric_features, MinMaxScaler())

2.1.1 Solo ohe encoder

In [ ]:
train_df, test_df = oh_encoder(scaled_train_df, scaled_test_df, ['protocol_type', 'flag', 'service'])

2.1.1.1 f_classif \
2.1.1.2 chi2 \
2.1.1.3 mutual_info_classif \
2.1.1.4 cfs \
2.1.1.5 rfe \
2.1.1.6 pca 

In [ ]:
pipeline(train_df, test_df)

2.1.2 Solo label encoder

In [ ]:
train_df, test_df = l_encoder(scaled_train_df, scaled_test_df, ['service', 'protocol_type', 'flag'])

2.1.2.1 f_classif \
2.1.2.2 chi2 \
2.1.2.3 mutual_info_classif \
2.1.2.4 cfs \
2.1.2.5 rfe \
2.1.2.6 pca 

In [ ]:
pipeline(train_df, test_df)

2.1.3 ohe + label

In [ ]:
train_df, test_df = oh_encoder(scaled_train_df, scaled_test_df, ['protocol_type', 'flag'])
train_df, test_df = l_encoder(train_df, test_df, ['service', ])

2.1.3.1 f_classif \
2.1.3.2 chi2 \
2.1.3.3 mutual_info_classif \
2.1.3.4 cfs \
2.1.3.5 rfe \
2.1.3.6 pca 

In [ ]:
pipeline(train_df, test_df)

2.2 Non scalato

In [ ]:
# MADONNA BOCCHINARA
unscaled_train_df = Dataset(starting_train_df, columns).get_label5()
unscaled_test_df = Dataset(starting_test_df, columns).get_label5()

2.2.1 Solo ohe encoder

In [ ]:
train_df, test_df = oh_encoder(unscaled_train_df, unscaled_test_df, ['protocol_type', 'flag', 'service'])

2.2.1.1 f_classif \
2.2.1.2 chi2 \
2.2.1.3 mutual_info_classif \
2.2.1.4 cfs \
2.2.1.5 rfe \
2.2.1.6 pca 

In [ ]:
pipeline(train_df, test_df)

2.2.2 Solo label encoder

In [ ]:
train_df, test_df = l_encoder(unscaled_train_df, unscaled_test_df, ['service', 'protocol_type', 'flag'])

2.2.2.1 f_classif \
2.2.2.2 chi2 \
2.2.2.3 mutual_info_classif \
2.2.2.4 cfs \
2.2.2.5 rfe \
2.2.2.6 pca 

In [ ]:
pipeline(train_df, test_df)

2.2.3 ohe + label

In [ ]:
train_df, test_df = oh_encoder(unscaled_train_df, unscaled_test_df, ['protocol_type', 'flag'])
train_df, test_df = l_encoder(train_df, test_df, ['service', ])

2.2.3.1 f_classif \
2.2.3.2 chi2 \
2.2.3.3 mutual_info_classif \
2.2.3.4 cfs \
2.2.3.5 rfe \
2.2.3.6 pca 

In [ ]:
pipeline(train_df, test_df)

2.3 Discretizzato

In [ ]:
train_df = Dataset(starting_train_df, columns).get_label5()
test_df = Dataset(starting_test_df, columns).get_label5()

discretizer = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='uniform')
X_train = train_df.drop(columns=['label'])
X_test = test_df.drop(columns=['label'])
X_train = discretizer.fit_transform(X_train)
X_test = discretizer.transform(X_test)

discretized_train_df = pd.concat([pd.DataFrame(X_train), train_df['label']], axis=1)
discretized_test_df = pd.concat([pd.DataFrame(X_test), test_df['label']], axis=1)

2.3.1 Solo one hot encoder

In [ ]:
train_df, test_df = oh_encoder(discretized_train_df, discretized_test_df, ['protocol_type', 'flag', 'service'])

2.1.3.1 f_classif \
2.1.3.2 chi2 \
2.1.3.3 mutual_info_classif \
2.1.3.4 cfs \
2.1.3.5 rfe \
2.1.3.6 pca 

In [ ]:
pipeline(train_df, test_df)

2.3.2 Solo label encoder

In [ ]:
train_df, test_df = l_encoder(discretized_train_df, discretized_test_df, ['service', 'protocol_type', 'flag'])

2.3.2.1 f_classif \
2.3.2.2 chi2 \
2.3.2.3 mutual_info_classif \
2.3.2.4 cfs \
2.3.2.5 rfe \
2.3.2.6 pca 

In [ ]:
pipeline(train_df, test_df)

1.3.4 ohe + label

In [ ]:
train_df, test_df = oh_encoder(discretized_train_df, discretized_test_df, ['protocol_type', 'flag'])
train_df, test_df = l_encoder(train_df, test_df, ['service', ])

1.3.4.1 f_classif \
1.3.4.2 chi2 \
1.3.4.3 mutual_info_classif \
1.3.4.4 cfs \
1.3.4.5 rfe \
1.3.4.6 pca 

In [ ]:
pipeline(train_df, test_df)